In [0]:
# Define Error Callbacks
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
# Create Kafka Connection Strings

# Connection String
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic

########### CHANGE CONFLUENT TOPIC NAME TO YOUR TOPIC NAME THAT YOU CREATE ########################

#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "pushing-p"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


# Kafka Admin Setup
admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})
#Kakfa Producer Setup.
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(1),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
# Delete Topic (if it exists)
try:
    topics =['pushing-p']
    fs = admin_client.delete_topics(topics, request_timeout=30)

    for topic, f in fs.items():
        try:
            f.result()  # The result itself is None
            print("Topic {} deleted".format(topic))
        except Exception as e:
            print("Failed to delete topic {}: {}".format(topic, e))
except Exception as e:
    print(e)
sleep(2)

Topic pushing-p deleted

In [0]:
# Simple Topic Setup
topic_list = []
topic_list.append(NewTopic("pushing-p", 1, 3))
futures = admin_client.create_topics(topic_list)

try:
    record_metadata = []
    for k, future in futures.items():
        print(f"type(k): {type(k)}")
        print(f"type(v): {type(future)}")
        print(future.result())

except KafkaError:
    # Decide what to do if produce request failed...
    print(traceback.format_exc())
    result = 'Fail'
finally:
    print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
None
finally

In [0]:
admin_database = "master"
admin_user = "gen10dbadmin"
admin_password  = "vbtwrEmgSG12mabBr9ReZkcPrrDbTR-Y"
admin_server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

database = "Pushing-P-DB"
table = "dbo.NJ_Cities"
user = "pushing_p"
password  = "t3stP@ssword"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

In [0]:
# Read from City Table in SQL Database
import pandas as pd
import pymssql

conn = pymssql.connect(server, user, password, database)

# Query select all rows from SQL tables to insert into their respective DataFrames
query = f'SELECT * FROM {table}'
city_info = pd.read_sql(query, conn)

# Sort alphabetically by County
city_info.sort_values(by = 'County', ascending=True, inplace=True)

# Convert to PySpark DataFrame
city_info = spark.createDataFrame(city_info)
display(city_info)

State County City Unemployment Rate (16 & Over) Median Household Income City Population Uninsured Population NUI, Under 6 years NUI, 6-18 years NUI, 19 to 25 years NUI, 26 to 34 years NUI, 35 to 44 years NUI, 45 to 54 years NUI, 55 to 64 years NUI, 65 years and older NUI, Men NUI, Women NUI, White NUI, African American NUI, American Indian & Alaska Natives NUI, Asians NUI, Native Hawaiians & Pacific Islanders NUI, Other Races geometry New Jersey Atlantic County Northfield 6.5 84464 7926 465 27 20 0 82 234 40 57 5 166 299 418 0 0 8 0 39 0103000000010000001A000000CE1951DA1BA552C01A4B581B63AF43402407EC6AF2A452C0B16A10E676AF4340799274CDE4A452C0637B2DE8BDAF434059897956D2A452C0C974E8F4BCAF43401DC6A4BF97A452C03126FDBD14B04340A93121E692A452C00F43AB9333B0434070067FBF98A452C01457957D57B04340D903ADC090A452C00C7217618AB043404F3DD2E0B6A352C0AA9B8BBFEDB143407FD8D2A3A9A352C016F4DE1802B243403A5B40683DA352C0A911FA997AB14340C493DDCCE8A252C05C6F9BA910B143408350DEC7D1A252C0F44E05DCF3B04340DBC01DA853A252C0632827DA55B04340D82C978DCEA152C023D74D29AFAF43405E2EE23B31A252C0958098840BAF43401C40BFEFDFA252C0658BA4DDE8AD4340EB6F09C03FA352C0C22D1F4949AD4340C6DE8B2FDAA352C05016BEBED6AD4340A8363811FDA352C05B04C6FA06AE4340531F48DE39A452C066BE839F38AE4340158E209562A452C0F7544E7B4AAE4340E481C8224DA452C01C2444F982AE4340B3B3E89D0AA552C0315D88D51FAF434025AFCE3120A552C0C5724BAB21AF4340CE1951DA1BA552C01A4B581B63AF4340 New Jersey Atlantic County Brigantine 5.900000095367432 75329 8685 322 11 18 33 130 62 26 42 0 173 149 268 1 0 17 0 20 0103000000010000003A0000000589EDEE019B52C0E333D93F4FB1434037195586719A52C04E97C5C4E6B143403EB2B96A9E9A52C02B2E8ECA4DB24340F0FACC599F9A52C0B6D9588979B243401A51DA1B7C9A52C07406465ED6B2434002F22554709A52C0419B1C3EE9B2434008043A93369A52C09EB4705985B34340705F07CE199A52C0FAD4B14AE9B34340F44E05DCF39952C0A5D7666325B443409526A5A0DB9952C0035ABA826DB44340DDEC0F94DB9952C0681EC022BFB44340EA944737C29952C061A6ED5F59B543401AA54BFF929952C03F36C98FF8B543402D043928619952C00D52F01472B74340C899266C3F9952C0AAB8718BF9B743401A6CEA3C2A9952C0A33B889D29B84340CAA7C7B60C9952C01281EA1F44B84340CDE67118CC9852C0AC36FFAF3AB8434013EF004F5A9852C09E3E027FF8B7434098F56228279852C092567C43E1B743402172FA7ABE9752C075711B0DE0B74340C53A55BE679752C0B6A1629CBFB74340349BC761309752C0F06B2409C2B74340791D71C8069752C0BC1DE1B4E0B74340876BB587BD9652C0C8B3CBB73EB843403C9F01F5669652C0D7135D177EB84340DAFF006BD59552C00629780AB9B84340DEC9A7C7B69552C02769FE98D6B84340FA0AD28C459552C0E5CE4C309CB9434098A1F144109552C0E659492BBEB94340367F9BD0DD9452C0232AD7A895B94340C43F6CE9D19452C0227024D060B94340E262768DE99452C08D83301BB2B84340D32EA699EE9452C0473CD9CD8CB843406D6FB724079552C0DCD7817346B843406A56FA8C089552C067E7519F43B843405987A3AB749552C0583673486AB74340EA20AF07939552C0836C59BE2EB74340ADDA3521AD9552C0C092AB58FCB64340C2BD326FD59552C09FAA4203B1B64340BC2363B5F99552C0ACAA97DF69B643405F28603B189652C0E55E605628B6434015376E313F9652C02A53CC41D0B54340BD158E72AC9652C0E4C58BD100B543408FA4B2B5309752C0499211B205B4434061FD9FC37C9752C0D0B7054B75B34340C9D24201809752C0FB47A89370B3434098ECE740E49752C0A98F7DAFDEB24340A61AAA47789852C013D8724307B24340676E8841C99852C0C86B616B91B14340A629029CDE9852C090300C5872B14340739590E6479952C05B31C204CBB04340EF537249CE9952C0F34CD674F5AF4340F4491F2FE19952C0D88BF66CD7AF4340170B43E4F49952C054E0641BB8AF43404D8F73C3019A52C0458338636DAF43401B7E37DDB29A52C02F88484DBBB043400589EDEE019B52C0E333D93F4FB14340 New Jersey Atlantic County Mullica Township 5.099999904632568 79155 5869 285 18 21 55 22 29 75 42 23 164 121 285 0 0 0 0 0 0103000000010000005C000000C217265305B152C08993FB1D8ACE434015AC71361DAF52C094A12AA6D2D14340EA211ADD41AB52C004ABEAE577D84340588B4F0130AB52C03BA6EECA2ED84340FF1F274C18AB52C0003ACC9717D84340D0B52FA017AB52C0B1A371A8DFD743402BD8463CD9AA52C041F0F8F6AED74340BD1AA034D4AA52C03196E99788D74340A9F3A8F8BFAA52C069A7E67283D74340FD4AE7C3B3AA52C0E76B96CB46D74340114F7633A3AA52C09207228B34D74340C51C041DADAA52C0B2BCAB1E30D74340B9347EE195AA52C0971C774A07D74340AA7F10C990AA52C00FD3BEB9BFD64340

In [0]:
# Create your producers
# Convert the rows of live_feed into a JSON key:value dictionary
results = city_info.toJSON().map(lambda j: json.loads(j)).collect()

# Send each row dictionary as a string message to Kafka
for i in range(len(results)):
    p.produce(confluentTopicName,json.dumps(results[i]))
    p.flush()
#     sleep(1)